In [7]:
import os
import re
import csv
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy.stats import gaussian_kde
import pandas as pd
from os import path
from trackmate import trackmate_peak_import

In [8]:
os.chdir("E:/Temperature")

In [9]:
liste_temperature = os.listdir()

In [10]:
def grep(liste_s,pattern):
    list_return=[]
    for s in (liste_s):
        res = ('\n'.join(re.findall(r'^.*%s.*?$'%pattern,s,flags=re.M)))
        if res != '':
            list_return.append(res)            
        
    return list_return

def dist(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

def get_outside_spots(x_base, y_base, xs, ys, radius):
    spots_outside_xs = list()
    spots_outside_ys = list()
    for i in range(0,len(xs)):
        if dist(x_base, y_base,xs[i],ys[i]) > radius:
            spots_outside_xs.append(xs[i])
            spots_outside_ys.append(ys[i])
    return spots_outside_xs,spots_outside_ys

def change_label(df, index, value):
    for j in df.loc[index]:
        df.set_value(index,"phase", value)

In [11]:
for temperature in (liste_temperature) :
    liste_lignee = os.listdir(temperature)
    #print(liste_lignee)
    for lignee in (liste_lignee) :
        liste_champ = os.listdir(temperature+"/"+lignee)
        #print(liste_champ)
        for champ in (liste_champ) :
            print ("\n\n\n ------------------------------------------------------\n\n\n")
            print (" Tu es dans : " + temperature + "\t"+ lignee + "\t"+ champ + "\n")
            list_cell= os.listdir(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/spots")
            #list_cell_GFP = grep(list_cell, "GFP")
            #list_cell_CFP = grep(list_cell, "CFP")
            
            # Creation d'un dataframe qui contient tous les resultats pour un champ
            res_champ = pd.DataFrame(columns=["cellule", "meta_entiere", "meta_partielle", "ana_entiere", "ana_partielle", \
                                              "debut_meta", "trans_ma", "fin_ana"])
            
            # Parcours des cellules 
            cpt=0
            for index in range (0,len(list_cell),2):
                
                print ("\n\nCellule en analyse : " + list_cell[index]+"\t"+list_cell[index+1]+"\n")
                
                cellule_en_mitose = input("Est-ce une mitose? ('o' ou 'n') :\t")
                while cellule_en_mitose!='o' and cellule_en_mitose!='n' :
                    cellule_en_mitose = input("Vous pouvez repeter la reponse!!! \n Est-ce une mitose? ('o' ou 'n'): \t")                
                if cellule_en_mitose == 'n':
                    print ("C'est la loose")
                else :
                    
                    # Creation des variables que l'on veut sauvegarder
                    #tmp = numpy.array()
                    num_cell = list_cell[index].split('_')[0]
                    meta_entiere = 0
                    meta_partielle = 0
                    ana_entiere = 0
                    ana_partielle = 0
                    debut_meta = 0
                    trans_ma = 0
                    fin_ana = 0
                    
                    print ("Youpie c'est une mitose!!! <3")
                    
                    # Ouverture des fichiers de spots
                    cfpSpots = trackmate_peak_import(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/spots/"+list_cell[index], False)
                    gfpSpots = trackmate_peak_import(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/spots/"+list_cell[index+1], False)
                    
                    labels_cfp = ["interphase"]*len(cfpSpots)
                    labels_gfp = ["interphase"]*len(gfpSpots)
                    cfpSpots["phase"] = labels_cfp
                    gfpSpots["phase"] = labels_gfp
                    meta_color = "r"
                    ana_color = "b"
                    radius = 0.3
                    an = np.linspace(0, 2*np.pi, 100)
                    circle_xs = radius *np.cos(an)
                    circle_ys = radius *np.sin(an)
                    """
                    for x in gfpSpots.index.levels[0]:
                        if x in cfpSpots.index and x in gfpSpots.index:
                            current_frame_poles = cfpSpots.loc[x]
                            current_frame_kts = gfpSpots.loc[x]
                            if len(current_frame_poles)>1:
                                alpha = 0.5
                                fig,ax = plt.subplots(figsize=(5,5))
                                ax.axis('equal')
                                ax.plot(circle_xs + current_frame_poles['x'].iloc[0], circle_ys + current_frame_poles['y'].iloc[0], "y--",alpha = alpha)
                                ax.plot(circle_xs + current_frame_poles['x'].iloc[1], circle_ys + current_frame_poles['y'].iloc[1], "y--",alpha = alpha)
                                spots_outside = (list(current_frame_kts['x']), list(current_frame_kts['y']))
                                for i in range(0,len(current_frame_poles['x'])):
                                    spots_outside = get_outside_spots(current_frame_poles['x'].iloc[i], current_frame_poles['y'].iloc[i],
                                                                      spots_outside[0],spots_outside[1],radius)
                                if len(spots_outside[0])==0:
                                    ax.scatter(current_frame_poles['x'],current_frame_poles['y'],alpha = alpha, c= ana_color)
                                    ax.scatter(current_frame_kts['x'],current_frame_kts['y'],alpha = alpha, c= "g")
                                    change_label(cfpSpots,x,"anaphase")
                                    change_label(gfpSpots,x,"anaphase")
                                else:
                                    ax.scatter(current_frame_poles['x'],current_frame_poles['y'],alpha = alpha, c= meta_color)
                                    ax.scatter(current_frame_kts['x'],current_frame_kts['y'],alpha = alpha, c= "g")
                                    change_label(cfpSpots,x,"metaphase")
                                    change_label(gfpSpots,x,"metaphase")
                                plt.title(x)
                                plt.show()
                                
                    """
                    metaphase_entiere = input ("Metaphase entiere connue ? ('o' ou 'n') :\t")
                    while metaphase_entiere!='n' and metaphase_entiere !='o' :
                        metaphase_entiere = input("Vous pouvez repeter la reponse!!! \n Metaphase entiere connue ?  ('o' ou 'n') : \t")
                    if metaphase_entiere == 'n':
                        meta_entiere = 0
                        print ("C'est la loose, pas de metaphase entiere")
                        metaphase_partielle = input("Metaphase partielle (uniquement fin metaphase) ? ('o' ou 'n') : \t")
                        while metaphase_partielle!='n' and metaphase_partielle !='o' :
                            metaphase_partielle = input("Vous pouvez repeter la reponse!!! \n Metaphase partielle connue (uniquement fin metaphase)?  ('o' ou 'n') : \t")
                       
                        if metaphase_partielle == 'o':
                            meta_partielle = 1
                            frame_milieu_metaphase = (input("Quelle frame pour le milieu de la metaphase ? : \t"))
                            ok=0
                            while ok == 0:
                                try:
                                    while int(frame_milieu_metaphase)<1 or int(frame_milieu_metaphase)>60:
                                        frame_milieu_metaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de la metaphase ? : \t"))
                                        if len(cfpSpots.loc[frame_milieu_metaphase]['x'])<=1 :
                                            frame_milieu_metaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (CFP)!!! Choisir une autre frame. \n Quelle frame pour le milieu de la metaphase ? : \t"))
                                        else :
                                            ok = 1
                                except: 
                                    frame_milieu_metaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de la metaphase ? : \t"))
                            
                                
                            frame_transition_m_a = (input("Quelle frame pour la transition metaphase-anaphase ? : \t"))
                            ok=0
                            while ok==0:
                                try :
                                    while int(frame_transition_m_a)<int(frame_milieu_metaphase) or int(frame_transition_m_a)>60:
                                        frame_transition_m_a = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la transition metaphase-anaphase ? : \t"))
                                        if len(cfpSpots.loc[frame_transition_m_a]['x'])<=1 and len(gfpSpots.loc[frame_transition_m_a]['x'])<=1 :
                                            frame_transition_m_a = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (CFP et GFP)!!! Choisir une autre frame. \n Quelle frame pour la transition metaphase-anaphase ? : \t"))
                                        else :
                                            ok = 1
                                except: 
                                    frame_transition_m_a = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la transition metaphase-anaphase ? : \t"))
                                                                      
                            
                            debut_meta = int(frame_milieu_metaphase)
                            trans_ma = int(frame_transition_m_a)
                            
                            anaphase_entiere =  input ("Anaphase entiere connue ? ('o' ou 'n') :\t")
                            while anaphase_entiere!='n' and anaphase_entiere !='o' :
                                anaphase_entiere = input("Vous pouvez repeter la reponse!!! \n Anaphase entiere connue ?  ('o' ou 'n') : \t")
                            if anaphase_entiere == 'n':
                                ana_entiere= 0
                                ana_partielle = 1
                                anaphase_partielle = 'o'
                                print ("C'est la loose, pas d'anaphase entiere")
                                frame_milieu_anaphase = (input("Quelle frame pour le milieu de l'anaphase ? : \t"))
                               
                                ok = 0
                                while ok == 0:
                                    try :
                                        while int(frame_milieu_anaphase)<int(frame_transition_m_a) or int(frame_milieu_anaphase)>60:
                                            frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                        if len(gfpSpots.loc[frame_milieu_anaphase]['x'])<=1 :
                                            frame_milieu_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                        else :
                                            ok = 1
                                    except:
                                        frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))

                                fin_ana = int(frame_milieu_anaphase)
                                
                                
                            else :
                                print ("Youpie anaphase entiere !!!")
                                ana_entiere = 1
                                ana_partielle=0
                                frame_fin_anaphase = input("Quelle frame pour la fin de l'anaphase ? : \t")
                                ok=0
                                while ok ==0 :
                                    try :
                                        while int(frame_fin_anaphase)<int(frame_transition_m_a) or int(frame_fin_anaphase)>60:
                                            frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                        if len(gfpSpots.loc[frame_fin_anaphase]['x'])<=1 :
                                            frame_fin_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                        else :
                                            ok = 1
                                    except :
                                        frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                fin_ana = int(frame_fin_anaphase)
                        
                        else :
                            meta_partielle=0
                            anaphase_partielle = input ("Anaphase partielle connue ? ('o' ou 'n') :\t")
                            while anaphase_partielle!='n' and anaphase_partielle !='o' :
                                anaphase_partielle = input("Vous pouvez repeter la reponse!!! \n Anaphase partielle connue ?  ('o' ou 'n') : \t")
                            if anaphase_partielle == 'o': 
                                ana_partielle = 1
                                ana_entiere = 0
                                frame_milieu_anaphase = (input("Quelle frame pour le milieu de l'anaphase ? : \t"))
                                ok = 0
                                while ok == 0 :
                                    try : 
                                        while int(frame_milieu_anaphase)<1 or int(frame_milieu_anaphase)>60:
                                            frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                        if len(gfpSpots.loc[frame_milieu_anaphase]['x'])<=1 :
                                            frame_milieu_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                        else :
                                            ok = 1
                                    except :
                                        frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                    
                                frame_fin_anaphase = (input("Quelle frame pour la fin de l'anaphase ? : \t"))
                                ok = 0
                                while ok ==0 :
                                    try :
                                        while int(frame_fin_anaphase)<int(frame_milieu_anaphase) or int(frame_fin_anaphase)>60:
                                            frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                        if len(gfpSpots.loc[frame_fin_anaphase]['x'])<=1 :
                                            frame_fin_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                        else :
                                            ok = 1
                                    except :
                                        frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                        
                                trans_ma = int(frame_milieu_anaphase)
                                fin_ana = int(frame_fin_anaphase)
                            #else : 
                                #input ("Est-ce vraiment une mitose ? ")
                        
                    else :
                        meta_entiere=1
                        print ("Youpie metaphase entiere !!!")
                        
                        frame_debut_metaphase = (input("Quelle frame pour le debut de la metaphase ? : \t"))
                        ok=0
                        while ok ==0:
                            try :
                                while int(frame_debut_metaphase)<1 or int(frame_debut_metaphase)>60:
                                    frame_debut_metaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le debut de la metaphase ? : \t"))
                                if len(cfpSpots.loc[frame_debut_metaphase]['x'])<=1 :
                                    frame_debut_metaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (CFP)!!! Choisir une autre frame. \n Quelle frame pour le debut de la metaphase ? : \t"))
                                else :
                                    ok = 1
                            except :
                                frame_debut_metaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le debut de la metaphase ? : \t"))
                        
                        debut_meta = int(frame_debut_metaphase)
                        
                        frame_transition_m_a = (input("Quelle frame pour la transition metaphase-anaphase ? : \t"))
                        ok=0
                        while ok == 0:
                            try :
                                while int(frame_transition_m_a)<int(frame_debut_metaphase) or int(frame_transition_m_a)>60:
                                    frame_transition_m_a = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la transition metaphase-anaphase ? : \t"))
                                if len(cfpSpots.loc[frame_transition_m_a]['x'])<=1 and len(gfpSpots.loc[frame_transition_m_a]['x'])<=1 :
                                    frame_transition_m_a = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (CFP et GFP)!!! Choisir une autre frame. \n Quelle frame pour la transition m-a ? : \t"))
                                else :
                                    ok = 1
                            except : 
                                frame_transition_m_a = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la transition metaphase-anaphase ? : \t"))
             
                            
                        trans_ma=int(frame_transition_m_a)
                    
                        anaphase_entiere =  input ("Anaphase entiere connue ? ('o' ou 'n') :\t")
                        ok = 0 
                        while ok == 0:
                            try :
                                while anaphase_entiere!='n' and anaphase_entiere !='o' :
                                    anaphase_entiere = input("Vous pouvez repeter la reponse!!! \n Anaphase entiere connue ?  ('o' ou 'n') : \t")
                                ok = 1
                            except :
                                anaphase_entiere = input("Vous pouvez repeter la reponse!!! \n Anaphase entiere connue ?  ('o' ou 'n') : \t")
                                
                        if anaphase_entiere == 'n':
                            ana_entiere = 0
                            ana_partielle = 1
                            anaphase_partielle = 'o'
                            print ("C'est la loose, pas d'anaphase entiere")
                            frame_milieu_anaphase = (input("Quelle frame pour le milieu de l'anaphase ? : \t"))
                            ok = 0
                            while ok == 0:
                                try : 
                                    while int(frame_milieu_anaphase)<int(frame_transition_m_a) or int(frame_milieu_anaphase)>60:
                                        frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                    if len(gfpSpots.loc[frame_milieu_anaphase]['x'])<=1 :
                                        frame_milieu_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                    else :
                                        ok = 1
                                except :
                                    frame_milieu_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour le milieu de l'anaphase ? : \t"))
                                    
                                
                            fin_ana = int(frame_milieu_anaphase)
                        
                        else :
                            ana_entiere = 1
                            ana_partielle = 0
                            print ("Youpie anaphase entiere !!!")
                            frame_fin_anaphase = (input("Quelle frame pour la fin de l'anaphase ? : \t"))
                            ok = 0
                            while ok == 0:
                                try:
                                    while int(frame_fin_anaphase)<int(frame_transition_m_a) or int(frame_fin_anaphase)>60:
                                        frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                    if len(gfpSpots.loc[frame_fin_anaphase]['x'])<=1 :
                                        frame_fin_anaphase = (input("CETTE FRAME NE POSSEDE PAS DEUX SPOTS (GFP)!!! Choisir une autre frame. \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                    else :
                                        ok = 1
                                except:
                                    frame_fin_anaphase = (input("Vous pouvez repeter la reponse!!! \n Quelle frame pour la fin de l'anaphase ? : \t"))
                                     
                            fin_ana=int(frame_fin_anaphase)
                        
           
                    res_champ.loc[cpt] = ({"cellule":num_cell, "meta_entiere":meta_entiere, "meta_partielle":meta_partielle, "ana_entiere":ana_entiere,\
                     "ana_partielle":ana_partielle, "debut_meta":debut_meta, "trans_ma":trans_ma, "fin_ana":fin_ana})
                    cpt = cpt+1
                


            # sauvegarde du fichier
            res_champ.to_csv(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/results.csv", sep=';')




 ------------------------------------------------------



 Tu es dans : 28C	102	28C_102_1



Cellule en analyse : 22_CFP.xml	22_GFP.xml

Est-ce une mitose? ('o' ou 'n') :	o
Youpie c'est une mitose!!! <3
Metaphase entiere connue ? ('o' ou 'n') :	n
C'est la loose, pas de metaphase entiere
Metaphase partielle (uniquement fin metaphase) ? ('o' ou 'n') : 	7
Vous pouvez repeter la reponse!!! 
 Metaphase partielle connue (uniquement fin metaphase)?  ('o' ou 'n') : 	o
Quelle frame pour le milieu de la metaphase ? : 	7
Quelle frame pour la transition metaphase-anaphase ? : 	7
Anaphase entiere connue ? ('o' ou 'n') :	n
C'est la loose, pas d'anaphase entiere
Quelle frame pour le milieu de l'anaphase ? : 	18


Cellule en analyse : 293_CFP.xml	293_GFP.xml

Est-ce une mitose? ('o' ou 'n') :	o
Youpie c'est une mitose!!! <3
Metaphase entiere connue ? ('o' ou 'n') :	n
C'est la loose, pas de metaphase entiere
Metaphase partielle (uniquement fin metaphase) ? ('o' ou 'n') : 	o
Quelle frame pour le mil

In [10]:
res_champ

,cellule,meta_entiere,meta_partielle,ana_entiere,ana_partielle,debut_meta,trans_ma,fin_ana
0,110,0,1,1,0,1,1,20
1,95,0,0,0,0,0,0,0


# Calcul

In [13]:
def calcul_duree (frame1, frame2, intervalle):
    return ((frame2-frame1)*intervalle/60)

"""
cell_gfp = fichier gfp ouvert
"""
def calcul_vitesse_anaphase (cell, frame1, frame2, intervalle):
    if len(cell.loc[frame1]['x'])>1 and len(cell.loc[frame2]['x'])>1:
        spot1x = cell.loc[frame1]['x'].iloc[0]
        spot1y = cell.loc[frame1]['y'].iloc[0]
        spot2x = cell.loc[frame1]['x'].iloc[1]
        spot2y = cell.loc[frame1]['y'].iloc[1]
        dist1  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)
        spot1x = cell.loc[frame2]['x'].iloc[0]
        spot1y = cell.loc[frame2]['y'].iloc[0]
        spot2x = cell.loc[frame2]['x'].iloc[1]
        spot2y = cell.loc[frame2]['y'].iloc[1]
        dist2  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)

        return ((dist2-dist1)/(frame2-frame1))
    
    else:
        return(-1)
    
def calcul_vitesse_metaphase (SPB, kinetochore, frame1, frame2, intervalle):
    if len(SPB.loc[frame1]['x'])>1 and len(SPB.loc[frame2]['x'])>1:
        spot1x = SPB.loc[frame1]['x'].iloc[0]
        spot1y = SPB.loc[frame1]['y'].iloc[0]
        spot2x = SPB.loc[frame1]['x'].iloc[1]
        spot2y = SPB.loc[frame1]['y'].iloc[1]
        dist1  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)
        spot1x = SPB.loc[frame2]['x'].iloc[0]
        spot1y = SPB.loc[frame2]['y'].iloc[0]
        spot2x = SPB.loc[frame2]['x'].iloc[1]
        spot2y = SPB.loc[frame2]['y'].iloc[1]
        dist2  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)

        return ((dist2-dist1)/(frame2-frame1))
    
    else if len(SPB.loc[frame1]['x'])>1 and len(kinetochore.loc[frame2]['x'])>1:
        spot1x = SPB.loc[frame1]['x'].iloc[0]
        spot1y = SPB.loc[frame1]['y'].iloc[0]
        spot2x = SPB.loc[frame1]['x'].iloc[1]
        spot2y = SPB.loc[frame1]['y'].iloc[1]
        dist1  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)
        spot1x = kinetochore.loc[frame2]['x'].iloc[0]
        spot1y = kinetochore.loc[frame2]['y'].iloc[0]
        spot2x = kinetochore.loc[frame2]['x'].iloc[1]
        spot2y = kinetochore.loc[frame2]['y'].iloc[1]
        dist2  = np.sqrt((spot2x-spot1x)**2 + (spot2y-spot1y)**2)

        return ((dist2-dist1)/(frame2-frame1))
        
    
    else:
        return(-1)
    

In [27]:
# Calcul à partir du tableau
for temperature in (liste_temperature) :
    liste_lignee = os.listdir(temperature)
    #print(liste_lignee)
    for lignee in (liste_lignee) :
        liste_champ = os.listdir(temperature+"/"+lignee)
        #print(liste_champ)
        for champ in (liste_champ) :
            
            result_champ = pd.read_csv(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/results.csv", sep=";") 
            
            result_champ["duree_meta"]   = 0
            result_champ["vitesse_meta"] = 0
            result_champ["duree_ana"]    = 0
            result_champ["vitesse_ana"]  = 0
            
            
            for index in range (len(result_champ)):
                #print (result_champ.iloc[index]["ana_partielle"])
                # CFP = SPB
                # GFP = Kinetochore
                cell_gfp = trackmate_peak_import (temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/spots/"+str(int(result_champ.iloc[index]["cellule"]))+"_GFP.xml")
                cell_cfp = trackmate_peak_import (temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/spots/"+str(int(result_champ.iloc[index]["cellule"]))+"_CFP.xml")
                
                
                ## Calcul pour la metaphase
                if result_champ.loc[index,"meta_entiere"] == 1 : 
                    # Calcul duree meta
                    result_champ.loc[index,"duree_meta"] = calcul_duree(result_champ.iloc[index]["debut_meta"],result_champ.iloc[index]["trans_ma"] , 60)
                    # Calcul vitesse meta
                    result_champ.loc[index,"vitesse_meta"] = calcul_vitesse_metaphase(cell_cfp,cell_gfp, result_champ.iloc[index]["debut_meta"],result_champ.iloc[index]["trans_ma"] , 60)
                
                if result_champ.loc[index,"meta_partielle"] == 1 : 
                    # Calcul vitesse meta
                    result_champ.loc[index,"vitesse_meta"] = calcul_vitesse_metaphase(cell_cfp, cell_gfp, result_champ.iloc[index]["debut_meta"],result_champ.iloc[index]["trans_ma"] , 60)
                
                
                
                ## Calcul pour l'anaphase
                if result_champ.loc[index,"ana_entiere"] == 1 : 
                    # Calcul duree ana
                    result_champ.loc[index,"duree_ana"] = calcul_duree(result_champ.iloc[index]["trans_ma"],result_champ.iloc[index]["fin_ana"] , 60)
                    # Calcul vitesse ana
                    result_champ.loc[index,"vitesse_ana"] = calcul_vitesse_anaphase(cell_gfp, result_champ.iloc[index]["trans_ma"],result_champ.iloc[index]["fin_ana"] , 60)
                
                if result_champ.loc[index,"ana_partielle"] == 1 : 
                    # Calcul vitesse ana
                    result_champ.loc[index,"vitesse_ana"] = calcul_vitesse_anaphase(cell_gfp, result_champ.iloc[index]["trans_ma"],result_champ.iloc[index]["fin_ana"] , 60)
                
            # sauvegarde du fichier
            result_champ.to_csv(temperature+"/"+lignee+"/"+champ+"/X0_Y0_MITOSIS/Resultats_finaux.csv", sep=';')

NotADirectoryError: [WinError 267] Nom de répertoire non valide: 'Capture.JPG'

In [26]:
((result_champ))

,;cellule;meta_entiere;meta_partielle;ana_entiere;ana_partielle;debut_meta;trans_ma;fin_ana,duree_meta,vitesse_meta,duree_ana,vitesse_ana
0,0;110;1.0;0.0;0.0;1.0;1.0;15.0;23.0,0,0,0,0
1,1;131;1.0;0.0;0.0;1.0;12.0;24.0;54.0,0,0,0,0
2,2;132;1.0;0.0;1.0;0.0;1.0;12.0;30.0,0,0,0,0
3,3;25;1.0;0.0;1.0;0.0;12.0;15.0;45.0,0,0,0,0
